In [240]:
import numpy as np
import copy
from func import conflictcalculator, link_maker
from itertools import product, combinations
from collections import Counter

In [303]:
class game_node():
    def __init__(self, playfield, g = 0, f = 0, h = 0, rc = False, ccol = None, ccolval = 0) -> None:
        l = locals()
        [self.__setattr__(name, l[name]) for name in self.__init__.__code__.co_varnames if name != "self"]
    
    def calc_score(self):
        return self.g+self.f-self.h

    def __eq__(self, other):
        if (self.playfield == other.playfield) and (self.calc_score() == other.calc_score()) and (self.rc == other.rc) and (self.ccol == other.ccol):
            return True
        return False

    def __lt__(self, other): #is self less than other
        o1, o2 = self.calc_score(), other.calc_score()
        if o1 == o2:
            if self.g == other.g:
                if self.f == other.f:
                    if self.h == other.h:
                        if self.rc == other.rc:
                            if self.ccolval == other.ccolval:
                                return True
                            else:
                                return False # objects are equal
                        else:
                            return self.rc
                    else:
                        return self.h > other.h
                else:
                    return self.f < other.f
            else:
                return self.g < other.g
        else:
            return o1 < o2

    def __repr__(self):
        return str(self.playfield) + f" g = {self.g}, f = {self.f}, h = {self.h}, rc = {self.rc}, cc = {self.ccol},{self.ccolval} \n"

In [304]:
def list_replace_copy(list, idx, val):
    nl = copy.copy(list)
    nl[idx] = val
    return nl

In [345]:
size = 8
# init_playfield = np.random.randint(0, size, size)
# init_playfield = [-1, 1, -1, -1] #for now only support this
# init_playfield = [1, -1, 2, 0] #for now only support this
init_playfield = [-1, -1, -1, -1, -1, -1, -1, 0] #for now only support this

In [348]:
cc = conflictcalculator(size)
fringe = [game_node(init_playfield)] #make self-sorting list
full_set = {i for i in range(size)}
explored_nodes = list()
breaker = 0
while len(fringe) > 0:
    breaker += 1
    if breaker > 100:
        break
    op = fringe.pop()
    explored_nodes.append(op)
    if len([idx for idx,i in enumerate(op.playfield) if i == -1]) > 0:
        new_pos =  [idx for idx, i in enumerate(op.playfield) if i == -1][0]# in the future, order by num conflicts per column, consider row conflicts as more important
        # new_pos = np.random.choice([i for i,k in conflicts.items() if k == new_min_c])
        print(op, "operating on", [idx for idx, i in enumerate(op.playfield) if i == -1], new_pos)
    else:
        new_pos = op.ccol
        print(op)
    if new_pos is None:
        print("found", op.playfield, breaker)
        break
    for pot_col in range(size):
        pot_playspace = list_replace_copy(op.playfield, new_pos, pot_col)
        g = (sum([True for i in pot_playspace if i != -1]) - 1) * size

        open_columns = [idx for idx,i in enumerate(pot_playspace) if i == -1]
        closed_columns = full_set.difference(set(open_columns))
        h = 0
        conflicted_columns = []
        for c1, c2 in combinations(closed_columns,2):
            x = cc.r_calc(pot_playspace[c1], c1, size)
            if((pot_playspace[c2] in x[c1]) or (pot_playspace[c2] in x[c2])):
                conflicted_columns.append(c1)
                conflicted_columns.append(c2)
                h += 1
        
        full_spaces = set(product(full_set, open_columns))
        for full_col_idx in closed_columns:
            x = cc.r_calc(pot_playspace[full_col_idx], full_col_idx, size)
            broken_spaces = set()
            for r, c in full_spaces:
                if ((r in x[full_col_idx]) or (r in x[c])):
                    broken_spaces.add((r,c))
            full_spaces = full_spaces.difference(broken_spaces)
        f = len(full_spaces)
        rc = len(x:=([pot_playspace[i] for i in closed_columns])) != len(np.unique(x))
        # print(pot_playspace, g, f, h, "\n")
        colsplit = dict(Counter(conflicted_columns))
        # print(pot_playspace, conflicted_columns, colsplit)
        if len(colsplit)>0:
            for k,v in colsplit.items():
                new_node = game_node(pot_playspace, g, f, h, rc, ccol = k, ccolval=v)
                fringe.append(new_node) if ((new_node not in fringe) and (new_node not in explored_nodes)) else None
        else:
            new_node = game_node(pot_playspace, g, f, h, rc)
            fringe.append(new_node) if ((new_node not in fringe) and (new_node not in explored_nodes)) else None
        # print(x, "\n")
        #calculate g, f, h
    fringe = sorted(fringe)
    # fringe = sorted(fringe)
    # print(fringe)


[-1, -1, -1, -1, -1, -1, -1, 0] g = 0, f = 0, h = 0, rc = False, cc = None,0 
 operating on [0, 1, 2, 3, 4, 5, 6] 0
[7, -1, -1, -1, -1, -1, -1, 0] g = 8, f = 30, h = 1, rc = False, cc = 0,1 
 operating on [1, 2, 3, 4, 5, 6] 1
[7, -1, -1, -1, -1, -1, -1, 0] g = 8, f = 30, h = 1, rc = False, cc = 7,1 
 operating on [1, 2, 3, 4, 5, 6] 1
[0, -1, -1, -1, -1, -1, -1, 0] g = 8, f = 30, h = 1, rc = True, cc = 0,1 
 operating on [1, 2, 3, 4, 5, 6] 1
[0, -1, -1, -1, -1, -1, -1, 0] g = 8, f = 30, h = 1, rc = True, cc = 7,1 
 operating on [1, 2, 3, 4, 5, 6] 1
[0, 1, -1, -1, -1, -1, -1, 0] g = 16, f = 21, h = 2, rc = True, cc = 7,1 
 operating on [2, 3, 4, 5, 6] 2
[0, 1, 1, -1, -1, -1, -1, 0] g = 24, f = 14, h = 3, rc = True, cc = 7,1 
 operating on [3, 4, 5, 6] 3
[0, 1, 1, 2, -1, -1, -1, 0] g = 32, f = 9, h = 4, rc = True, cc = 7,1 
 operating on [4, 5, 6] 4
[0, 1, 1, 2, 0, -1, -1, 0] g = 40, f = 6, h = 6, rc = True, cc = 3,1 
 operating on [5, 6] 5
[0, 1, 1, 2, 0, 7, -1, 0] g = 48, f = 2, h = 6, 

In [349]:
link_maker(op.playfield)

'https://lichess.org/editor/2Q5/Q7/6Q1/4Q3/7Q/1Q6/3Q4/5Q2_w_-_-_0_1?color=white'